In [11]:
import pandas as pd
import numpy as np

# Load data
df1 = pd.read_csv("ca1-dataset.csv")
df2 = pd.read_csv("ca2-dataset.csv")

#find no variance data in ca2
cols_with_zero = [col for col in df2.columns if df2[col].nunique() == 1]

print(cols_with_zero)



['help', 'choice', 'string', 'number', 'point', 'helppct-up', 'recent8help', 'asymptoteA-up', 'asymptoteB-up']


In [12]:
#find no variance data in ca1
cols_with_zero_1 = [col for col in df1.columns if df1[col].nunique() == 1]

print(cols_with_zero_1)

['Avghelp', 'Avgchoice', 'Avgstring', 'Avgnumber', 'Avgpoint', 'Avghelppct-up', 'Avgrecent8help', 'AvgasymptoteA-up', 'AvgasymptoteB-up']


In [13]:
# Selecting the  Pknow-1 &2 & Pchange and calculating correlation
selected_cols = df2[['pknow-1','Pknow-2','pchange']]
correlation_matrix = selected_cols.corr()

print(correlation_matrix)

# print(df2.columns)

          pknow-1   Pknow-2   pchange
pknow-1  1.000000  0.424843 -0.981923
Pknow-2  0.424843  1.000000 -0.305216
pchange -0.981923 -0.305216  1.000000


In [14]:
# Selecting the  howmanywrong-up & wrongpct-up and calculating correlation
selected_cols = df2[['howmanywrong-up','wrongpct-up']]
correlation = selected_cols.corr()

print(correlation)

                 howmanywrong-up  wrongpct-up
howmanywrong-up         1.000000     0.553766
wrongpct-up             0.553766     1.000000


In [15]:
# Selecting the  timeSDnormed','timelast3SDnormed','timelast5SDnormed and calculating correlation
selected_cols = df2[['timeSDnormed','timelast3SDnormed','timelast5SDnormed']]
correlation = selected_cols.corr()

print(correlation)

                   timeSDnormed  timelast3SDnormed  timelast5SDnormed
timeSDnormed           1.000000           0.096334           0.107049
timelast3SDnormed      0.096334           1.000000           0.855967
timelast5SDnormed      0.107049           0.855967           1.000000


In [16]:
# Selecting the  'Prev3Count-up'&'Prev5Count-up'and calculating correlation
selected_cols = df2[['Prev3Count-up','Prev5Count-up']]
correlation = selected_cols.corr()

print(correlation)

               Prev3Count-up  Prev5Count-up
Prev3Count-up       1.000000       0.956364
Prev5Count-up       0.956364       1.000000


In [17]:
#get Dummy categorical columns
dummies = pd.get_dummies(df2[['prod', 'cell','lesson']], prefix=['dummy_prod', 'dummy_cell','dummy_lesson'])
# Add the 'ID' column to the dummy DataFrame for merging
dummies['Unique-id'] = df2['Unique-id']

# Merging 'pknow-1'& 'Pknow-2' into ca1 using Unique-id
merged_p = pd.merge(df1,df2[['Unique-id', 'pknow-1', 'Pknow-2','namea']], on='Unique-id', how='left')

# Merging 'dummy_prod', 'dummy_cell' into ca1 using Unique-id
merged1 = pd.merge(merged_p,dummies, on='Unique-id', how='left')


# print(merged_dummy)
# print(ca1_merged.columns)

In [18]:
# Feature5: Calculate the total number of steps
df2['totalstep'] = np.where(df2['wrongpct-up'] == 0, 
                            0, 
                            (df2['howmanywrong-up'] / df2['wrongpct-up']))
merged5 = pd.merge(merged1,df2[['totalstep','Unique-id']], on='Unique-id', how='left')

# Feature6: Calculate The total number of actions
df2['totalaction'] = np.where(df2['timeperact-up'] == 0, 
                            0, 
                            (df2['time'] / df2['timeperact-up']))
# merged6 = pd.merge(merged5,df2[['totalaction','Unique-id']], on='Unique-id', how='left')

# Feature7
df2['last5'] = df2['Prev5Count-up']*df2['timelast5SDnormed']
# merged7 = pd.merge(merged6,df2[['last5','Unique-id']], on='Unique-id', how='left')

# Feature8: if the wrong productions in the last five actions occupied a large percentage. 
df2['wrongper'] = np.where(df2['howmanywrong-up'] == 0, 
                            0, 
                            (df2['recent5wrong'] / df2['howmanywrong-up']))
# if 'recent5wrong' in df2.columns:
#     df2['wrongper'] = np.where(df2['howmanywrong-up'] == 0, 
#                                 0, 
#                                 (df2[' recent5wrong'] / df2['howmanywrong-up']))
# else:
#     print("Column 'recent5wrong' does not exist!")
# merged8 = pd.merge(merged7,df2[['wrongper','Unique-id']], on='Unique-id', how='left')

# Feature9: Calculate percentage of actions where the production was wrong in the overall number of notright
df2['numerator'] = df2['howmanywrong-up']*merged5['totalstep']
df2['percent_wrong'] = np.where(df2['notright'] == 0, 
                            0, 
                            (df2['numerator'] / df2['notright']))
# merged9 = pd.merge(merged8,df2[['percent_wrong','Unique-id']], on='Unique-id', how='left')

# Feature10: Calculate The total number of actions
df2['ratio5'] = np.where(df2['Prev5Count-up'] == 0, 
                            0, 
                            (df2['recent5wrong'] / df2['Prev5Count-up']))
# ca1_merged = pd.merge(merged9,df2[['ratio5','Unique-id']], on='Unique-id', how='left')



# Merge all at once
all_features = ['totalstep', 'totalaction', 'last5', 'wrongper', 'percent_wrong', 'ratio5', 'Unique-id']
merged_all = pd.merge(merged1, df2[all_features], on='Unique-id', how='left')

# print(df2.columns)

# print(merged_all.columns)

In [32]:
#CA2
import pandas as pd
from sklearn.metrics import (
    cohen_kappa_score,
    roc_auc_score,
    roc_curve,
)
from scipy.stats import spearmanr
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, cohen_kappa_score


def evaluate_dataset(dataset_path):
    dataset = pd.read_csv(dataset_path)


        # Create a dictionary to store group information
    group_dict = {}
    groups = np.array([])

    # Iterate through the dataset to create groups
    for index, row in dataset.iterrows():
        namea = row['namea']
        if namea not in group_dict:
            group_dict[namea] = index
        groups = np.append(groups, group_dict[namea])

    # Initialize the GroupKFold cross-validator
    group_kfold = GroupKFold(n_splits=10)

    # Define the target variable 'OffTask' and features (all columns except 'OffTask')
    X = dataset.drop(columns=['OffTask', 'Unique-id', 'namea'])  # Features
    y = dataset['OffTask']  # Target variable
    y = y.map({'N': 0, 'Y': 1})



     # Create lists to store evaluation metrics for each fold
    kappa_scores = []
    roc_auc_scores = []


# # Define parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2], 
#     'n_estimators': [50, 100, 200], 
#     'max_depth': [5, 6, 7],  
#     'alpha': [0.1, 1, 10],
#     'lambda': [0.1, 1, 10]
# }

# # Initialize classifier
# clf = xgb.XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='error',
#     use_label_encoder=False,
#     random_state=42
# )

# # Define custom scorer
# kappa_scorer = make_scorer(cohen_kappa_score)

# # Initialize GridSearchCV with custom scorer
# grid_clf = GridSearchCV(clf, param_grid, scoring=kappa_scorer, cv=5, n_jobs=-1)

# # Fit model
# grid_clf.fit(X, y)  # Ensure your feature matrix X and target vector y are appropriately defined

# # Get best parameters
# best_params = grid_clf.best_params_
# print(f"Best parameters: {best_params}")

# Initialize the parameters for the XGBClassifier

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'use_label_encoder': False,
        'learning_rate': 0.2,  # Reduced learning rate
        'n_estimators': 100,  # Increase n_estimators if you decrease learning rate
        'random_state': 10,
        'max_depth': 5,  # Limiting max_depth. You may want to optimize this using cross-validation
        'alpha': 0.1,  # L1 regularization
        'lambda': 1,  # L2 regularization
    }

    #feature selection
    # Create the XGBClassifier model
    clf_whole_dataset = xgb.XGBClassifier(**params)

    # Fit the classifier to the entire dataset for feature selection
    clf_whole_dataset.fit(X, y)

    # Get feature importances from the trained model
    feature_importances = clf_whole_dataset.feature_importances_

    # Sort features based on importance
    sorted_idx = np.argsort(feature_importances)[::-1]

    # Choose the top N features (for example, select top 10 features)
    top_n = 5
    selected_features = X.columns[sorted_idx[:top_n]]

    # Update your feature matrix X to include only selected features
    X_selected = X[selected_features]


    # Iterate through the folds using the reduced feature set
    for train_index, test_index in group_kfold.split(X_selected, y, groups):
        X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #  # Iterate through the folds using the reduced feature set
    #     for train_index, test_index in group_kfold.split(X_selected, y, groups):
    #         X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
    #         y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Create a DecisionTreeClassifier with specified hyperparameters
        clf = xgb.XGBClassifier(**params)
        # Use early stopping
        eval_set = [(X_train, y_train), (X_test, y_test)]
        clf.fit(X_train, y_train, eval_metric="error", eval_set=eval_set, early_stopping_rounds=50, verbose=False)

        # Use early stopping
        eval_set = [(X_train, y_train), (X_test, y_test)]
        clf.fit(X_train, y_train, eval_metric="error", eval_set=eval_set, early_stopping_rounds=50, verbose=False)


        # Fit the classifier to the training data
        clf.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = clf.predict(X_test)

        # Calculate Cohen's Kappa
        kappa = cohen_kappa_score(y_test, y_pred)
        kappa_scores.append(kappa)

        # Calculate ROC AUC
        roc_auc = roc_auc_score(y_test, y_pred)
        roc_auc_scores.append(roc_auc)


# Calculate the mean values of the evaluation metrics across all folds
    mean_kappa_score = np.mean(kappa_scores)
    mean_roc_auc = np.mean(roc_auc_scores)

    return mean_kappa_score, mean_roc_auc

# # print("Mean Cohen's Kappa Score:", mean_kappa_score)
# print("Mean ROC AUC:", mean_roc_auc)
# print("Mean kappa:", mean_kappa_score)



# Evaluate the first dataset
kappa_dataset1, roc_auc_dataset1 = evaluate_dataset('ca1-dataset_new.csv')

# Evaluate the second dataset
kappa_dataset2, roc_auc_dataset2 = evaluate_dataset('ca1-dataset.csv')

# Print and compare the metrics
print(f"Cohen's Kappa for new Dataset: {kappa_dataset1}")
print(f"Cohen's Kappa for CA1 Dataset: {kappa_dataset2}")
print(f"ROC AUC for new Dataset: {roc_auc_dataset1}")
print(f"ROC AUC for CA1 Dataset: {roc_auc_dataset2}")

# Compare Kappa
if kappa_dataset1 > kappa_dataset2:
    print("new Dataset has a higher Cohen's Kappa score.")
elif kappa_dataset1 < kappa_dataset2:
    print("Dataset 2 has a higher Cohen's Kappa score.")
else:
    print("Both datasets have the same Cohen's Kappa score.")

# Compare ROC AUC
if roc_auc_dataset1 > roc_auc_dataset2:
    print("new Dataset has a higher ROC AUC score.")
elif roc_auc_dataset1 < roc_auc_dataset2:
    print("Dataset 2 has a higher ROC AUC score.")
else:
    print("Both datasets have the same ROC AUC score.")



Cohen's Kappa for new Dataset: 0.3450852848009799
Cohen's Kappa for CA1 Dataset: 0.2967050796364549
ROC AUC for new Dataset: 0.6370896898042206
ROC AUC for CA1 Dataset: 0.6267461461873862
new Dataset has a higher Cohen's Kappa score.
new Dataset has a higher ROC AUC score.
